In [ ]:
from utils import *
from datasets import load_dataset
path_labels = "../datasets/Norway/train/annotations/xmls/"
path_imgs = "../datasets/Norway/train/images/"

create_metadata(path_imgs, path_labels)
dataset = load_dataset(path_imgs, split="train[:50]")